In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
import cv2

In [2]:
def load_data(path,files):
    import gzip
    import numpy as np
    paths = [path+ each for each in files ]
    with gzip.open(paths[0], 'rb') as lbpath:
        train_labels = np.frombuffer(lbpath.read(), np.uint8, offset=8)
    with gzip.open(paths[1], 'rb') as impath:
        train_images = np.frombuffer(impath.read(), np.uint8, offset=16).reshape(len(train_labels),28,28)
    with gzip.open(paths[2], 'rb') as lbpath:
        test_labels = np.frombuffer(lbpath.read(), np.uint8, offset=8)
    with gzip.open(paths[3], 'rb') as impath:
        test_images = np.frombuffer(impath.read(), np.uint8, offset=16).reshape(len(test_labels), 28, 28)
    return (train_labels,train_images), (test_labels,test_images)
path = '../datasets/fashion-mnist/'
files = ['train-labels-idx1-ubyte.gz','train-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz',  't10k-images-idx3-ubyte.gz']


In [3]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [4]:
(y_train_all,x_train_all),(y_test,x_test) = load_data(path, files)
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

print(x_train_all.shape)
print(y_train_all.shape)
print(x_test.shape)
print(y_test.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled=scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)
x_valid_scaled=scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [5]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32,kernel_size=3,padding="same",activation='relu',input_shape=(28,28,1)))
model.add(keras.layers.Conv2D(filters=32,kernel_size=3,padding="same",activation="relu"))
model.add(keras.layers.MaxPool2D(pool_size=2))

model.add(keras.layers.Conv2D(filters=64,kernel_size=3,padding="same",activation='relu'))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"))
model.add(keras.layers.MaxPool2D(pool_size=2))
          
model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding="same",activation='relu'))
model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"))
model.add(keras.layers.MaxPool2D(pool_size=2))
          
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
          
model.add(keras.layers.Dense(10,activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
             optimizer='sgd',metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 128)         7

In [ ]:
logdir = './cnn-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"fashion_mnist_model.h5")
callbacks=[
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)
]

history = model.fit(x_train_scaled,y_train,epochs=10,
                   validation_data=(x_valid_scaled,y_valid),
                   callbacks=callbacks)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
34656/55000 [=================>............] - ETA: 5s - loss: 0.9469 - accuracy: 0.6683

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
    
plot_learning_curves(history)

In [21]:
model.evaluate(x_test_scaled,y_test)

313/313 [==============================] - 1s 5ms/step - loss: 0.3086 - accuracy: 0.8850


[0.3086146116256714, 0.8849999904632568]